# HCMS at SemEval-2020 Task 9: A Neural Approach to Sentiment Analysis of Code-Mixed Texts
_[Aditya Srivastava](https://www.github.com/IamAdiSri), [V. Harsha Vardhan](https://www.github.com/talent404)_

The preprint can be viewed on [arXiv](https://arxiv.org/abs/2007.12076).

## Task Description

The task description and the accompanying data can be found [here](https://competitions.codalab.org/competitions/20654).

## Requirements

The code has been written in `Python 3.6.9`. Other requirements have been listed in `requirements.txt`.
All data files are to be placed in the root folder of this repository.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/'
!pip install Keras==2.8                                                                                                                                                            
!pip install Keras-Applications==1.0.8                                                                                                                                                   
!pip install Keras-Preprocessing==1.1.0                                                                                                                                                  
!pip install keras-self-attention==0.42.0
!pip install scikit-learn==0.22.2 
!pip install numpy==1.18.1
!pip install pandas==0.25.0
!pip install emoji==0.5.4
!pip install tqdm==4.43.0


/content/drive/MyDrive
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 41 kB 409 kB/s 
  Attempting uninstall: Keras-Preprocessing
    Found existing installation: Keras-Preprocessing 1.1.2
    Uninstalling Keras-Preprocessing-1.1.2:
      Successfully uninstalled Keras-Preprocessing-1.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras-preprocessing>=1.1.1, but you have keras-preprocessing 1.1.0 which is incompatible.
  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-py3-none-any.whl size=17294 sha256=15564975157529e5f49dc0f410fdb1a677acfe55d41f9091091318fd7239a133
  Stored in directory: /root/.cache/pip/wheels/fa/86/d1/e8a21c514fa99dde9db4b07d28e817fed9212b0c49

     |████████████████████████████████| 10.4 MB 5.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires pandas>=1.0, but you have pandas 0.25.0 which is incompatible.
google-colab 1.0.0 requires pandas>=1.1.0; python_version >= "3.0", but you have pandas 0.25.0 which is incompatible.
fbprophet 0.7.1 requires pandas>=1.0.4, but you have pandas 0.25.0 which is incompatible.
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-py3-none-any.whl size=42192 sha256=79a29f3fe891518ff8429d2b7a5adb20ce7f597bd60a17ddd11e24ee599eefee
  Stored in directory: /root/.cache/pip/wheels/f6/65/82/d742fe456cd8aa21ffe6c4c1eaeedf3c2d430689811bf328e1
Successful

In [ ]:
%cd '/content/drive/MyDrive/hcms-semeval20/'

/content/drive/MyDrive/hcms-semeval20


In [ ]:
from tqdm import tqdm_notebook as tqdm

import itertools
import emoji
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D
from keras_self_attention import SeqSelfAttention
from keras.layers import Flatten, Dropout, Dense

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords 
import re,string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
!pip install pyspellchecker
from spellchecker import SpellChecker
!pip install emoji
import emoji
import pandas as pd
!pip install googletrans
from googletrans import Translator
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from nltk.corpus import wordnet
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#google translate
translator = Translator(service_urls =['translate.google.com'])

#pyspellchecker
spell = SpellChecker()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


     |████████████████████████████████| 2.7 MB 5.1 MB/s 
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 1.3 MB 14.1 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=833a0cc323c13ee939e505d8ff42b236e37524dd5d537d454ab41edb8b4d96fd
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptr

## Data Pipeline

### Loading and Parsing

In [ ]:
def parse_lines(lines):
    u = [] # uids
    t = [] # tokens
    l = [] # token labels
    s = [] # sentiment labels
    max_length = 0

    print("Parsing lines from file...")
    for i, line in tqdm(enumerate(lines), total=len(lines)):
        line = line.strip().split('\t')
        if line[0]=='meta':
            if i!=0:
                u.append(buffer_id)
                t.append(buffer_tokens)
                l.append(buffer_labels)
                s.append(buffer_sentiment)
                if len(buffer_tokens) > max_length:
                    max_length = len(buffer_tokens)
            buffer_id = line[1]
            try:
                buffer_sentiment = line[2]
            except:
                buffer_sentiment = ''
            buffer_tokens = []
            buffer_labels = []
        else:
            buffer_tokens.append(line[0])
            try:
                buffer_labels.append(line[1])
            except:
                buffer_labels.append('')

    u.append(buffer_id)
    t.append(buffer_tokens)
    l.append(buffer_labels)
    s.append(buffer_sentiment)
    if len(buffer_tokens) > max_length:
        max_length = len(buffer_tokens)

    num_samples = len(u)
    
    return u, t, l, s, max_length

In [ ]:
train = open('train_14k_split_conll.txt', encoding='utf8').readlines()
valid = open('dev_3k_split_conll.txt', encoding='utf8').readlines()
test = open('Hindi_test_unalbelled_conll_updated.txt', encoding='utf8').readlines()

u_train, t_train, l_train, s_train, max_length = parse_lines(train)
u_dev, t_dev, l_dev, s_dev, max_length_dev = parse_lines(valid)
u_test, t_test, l_test, s_test, max_length_test = parse_lines(test)

Parsing lines from file...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Parsing lines from file...



Parsing lines from file...


### Cleaning

In [ ]:
# source: https://en.wikipedia.org/wiki/List_of_emoticons
def load_dict_smileys():
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

# source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
def load_dict_contractions():
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }


def tweet_cleaning_for_sentiment_analysis(tweet):
    # lower case
    tweet = tweet.lower()
        
    # replace contractions
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    # standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    
    # replace emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    
    # demojize emojis
    tweet = emoji.demojize(tweet)
    
    # other cleaning
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    # replace duplicate characters
    tweet = re.sub(r"(.)\1{2,}", r'\1\1', tweet)

    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub(r"[www.][a-zA-Z0-9_]+[.com]", "", tweet)

    tweet = re.sub("(@[A-Za-z0-9_]+)","", tweet)

    return tweet

def clean(t,l):
    for i in range(len(t)):
        temp = tweet_cleaning_for_sentiment_analysis(' '.join(t[i])).split(' ')
        t[i] = []
        j=0
        while j<len(temp):
            t[i].append(temp[j])
            j+=1
        
    return t, l


def lemmatization(sent):
    lemmatize=WordNetLemmatizer()
    sentence_after_lemmatization=[]
    for word,tag in pos_tag(word_tokenize(sent)):
        if(tag[0:2]=="NN"):
            pos='n'
        elif(tag[0:2]=="VB"):
            pos='v'
        else:
            pos='a'
        lem=lemmatize.lemmatize(word,pos)
        sentence_after_lemmatization.append(lem)
    st=""
    for i in sentence_after_lemmatization:
        if(i!="be" and i!="is" and len(i)!=1):
            st=st+" "+i
    #print("lemi",st)
    c=0
    list_text=st.split()
    flag=0
    new_st=""
    for i in list_text:
        temp=i
        if(flag==1):
            flag=0
            continue
        if(i=="not" and (c+1)<len(list_text)):
            for syn in wordnet.synsets(list_text[c+1]):
                antonyms=[]
                for l in syn.lemmas():
                    #print(l)
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                        #print(antonyms)
                        temp=antonyms[0]
                        flag=1
                        break
                if(flag==1):
                    break
        new_st=new_st+" "+temp
        c+=1
    #print(new_st)
    return new_st



In [ ]:
t_train, l_train = clean(t_train, l_train)
t_dev, l_dev = clean(t_dev, l_dev)
t_test, l_test = clean(t_test, l_test)

[['Eng',
  'O',
  'Eng',
  'Eng',
  'Eng',
  'Eng',
  'Hin',
  'Hin',
  'O',
  'Eng',
  'Eng',
  'O',
  'Hin',
  'Eng',
  'Eng',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'O',
  'Hin',
  'O',
  'Eng',
  'Eng',
  'O',
  ''],
 ['O',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'O',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'EMT',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Eng',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'O',
  'Eng',
  'O',
  'Eng',
  'O',
  'Eng',
  ''],
 ['O',
  'Eng',
  'Eng',
  'Eng',
  'Eng',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'O',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'O',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Eng',
  'O',
  'Eng',
  'O',
  'Eng',
  'O',
  'Hin',
  'O',
  'Hin',
  ''],
 ['O',
  'Hin',
  'O',
  'Hin',
  'Hin',
  'Hin',
  'Eng',
  'Hin',
  'Hin',
  'Hin',
  'Eng',
  'Hin',
  'Hin',
  'Eng',
  'Eng',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  'Hin',
  ''],
 ['O',
  'Eng',
  'O',

### Preprocessing

In [ ]:
tok_w = Tokenizer(char_level=False,lower=True,oov_token='UNK')
tok_w.fit_on_texts(t_train) 

MAX_LEN = 60
trainInput_w = pad_sequences(tok_w.texts_to_sequences(t_train),
                          maxlen=MAX_LEN, padding="post")

valInput_w = pad_sequences(tok_w.texts_to_sequences(t_dev) ,
                          maxlen=MAX_LEN, padding="post")
testInput_w = pad_sequences(tok_w.texts_to_sequences(t_test),
                          maxlen=MAX_LEN, padding="post")

In [ ]:
s_train

['neutral',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neu

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(s_train)

trainLabels = to_categorical(le.transform(s_train))
valLabels = to_categorical(le.transform(s_dev))

## Classifier Pipeline

### Model Architecture, Optimizer, Loss Function and Hyperparameters

In [ ]:
max_features = len(tok_w.word_index)
maxlen = 60
embedding_size = 100

# Convolution
kernel_size = 5
filters = 128
pool_size = 4

model = Sequential()
model.add(Embedding(max_features+1, embedding_size, input_length=maxlen))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [ ]:
model.fit([trainInput_w], trainLabels,
          batch_size=32,
          validation_data=(valInput_w,valLabels),
          epochs=1)

438/438 [==============================] - 35s 77ms/step - loss: 0.8964 - accuracy: 0.5600 - val_loss: 0.8319 - val_accuracy: 0.6103


## Test

### Make Predictions on Test Set

In [ ]:
predictions = model.predict([testInput_w])
predictions = np.argmax(predictions,axis=-1)

# write predictions to file
with open('preds.txt', 'w') as out:
    out.write('Uid,Sentiment')
    for i, uid in enumerate(u_test):
        if predictions[i] == 0:
            sentiment = 'negative'
        elif predictions[i] == 1:
            sentiment = 'neutral'
        else:
            sentiment = 'positive'
        out.write("\n%s,%s"%(uid, sentiment))

### Evaluate Test Predictions

In [ ]:
# load correct labels
test = pd.read_csv('test_labels_hinglish.txt')
# load predictions
preds = pd.read_csv('preds.txt')

# compute evaluation metrics
results = {'preds': classification_report(test['Sentiment'], 
                                          preds['Sentiment'], 
                                          labels=['positive', 'neutral', 'negative'], 
                                          output_dict=True, digits=6)}

In [ ]:
# format and print scores
formatted_results = [['model', 'precision', 'recall', 'accuracy', 'f1-score']]
for ki in results.keys():
    scores = results[ki]['macro avg']
    model = [ki, scores['precision'], scores['recall'], results[ki]['accuracy'], scores['f1-score']]
    formatted_results.append(model)
    
formatted_results = pd.DataFrame(formatted_results[1:], columns=formatted_results[0])
print(formatted_results)

   model  precision    recall  accuracy  f1-score
0  preds   0.681706  0.680411  0.676333  0.680694
